# Kenn Experiments - Plots and Statistics
Run this notebook only after all the tests have been run. 

In [ ]:
from model import *
from pre_elab import *
from evaluation_functions import *
import settings as s
import pickle

# Loading results

In [ ]:
history_inductive = []
history_transductive = []

for i in range(1,7):
    with open('results/e2e/results_inductive_100runs_{}layers'.format(i),'rb') as input:
        history_inductive.append(pickle.load(input))
    
    with open('results/e2e/results_transductive_100runs_{}layers'.format(i),'rb') as input:
        history_transductive.append(pickle.load(input))

# Print stats

In [ ]:
def print_stats(history_all):
    n_layers_tried = len(history_all)
    for n_layers, history in zip(range(1, n_layers_tried + 1), history_all):
        print("============ KE layers: {} ============".format(n_layers))
        
        means, stds, means_kenn, stds_kenn, means_deltas, stds_deltas = get_means_and_stds(history)
        p_vals = make_t_test(history)
        for i, key in enumerate(history.keys()):
            print("== {}% ==".format(key))
            print("Mean Test Accuracy:\tNN = {:8.6f}; KENN = {:8.6f}".format(
                means[i], means_kenn[i]))
            print("Test Accuracy std:\tNN = {:8.6f}; KENN = {:8.6f}".format(
                stds[i], stds_kenn[i]))
            print("\t\t\tDeltas Mean = {:8.6f}".format(means_deltas[i]))
            print("\t\t\tDeltas Std = {:8.6f}".format(stds_deltas[i]))
            print("\t\t\tright tailed p-value: {}".format(p_vals[i]))
            print()

In [ ]:
print_stats(history_inductive)

In [ ]:
print_stats(history_transductive)

# Plot deltas
Improvements in terms of accuracy obtained by the different methods (KENN with 3 KEs layers, SBR, and RNM)

In [ ]:
def plot_deltas(history, ax, barwidth=0.3, title='', other_deltas='', confidence_level=0.95):
    assert(other_deltas == '' or other_deltas == 'i' or other_deltas == 't')
    _, _, _, _, means_deltas, _ = get_means_and_stds(history)


    results_NN_Marra_i = np.array([0.645, 0.674, 0.707, 0.717, 0.723])
    results_SBR_i = np.array([0.650, 0.682, 0.712, 0.719, 0.726])
    results_RNM_i = np.array([0.685, 0.709, 0.726, 0.726, 0.732])

    results_NN_Marra_t = np.array([0.640, 0.667, 0.695, 0.708, 0.726])
    results_SBR_t = np.array([0.703, 0.729, 0.747, 0.764, 0.780])
    results_RNM_t = np.array([0.708, 0.735, 0.753, 0.766, 0.780])

    deltas_SBR_i = results_SBR_i - results_NN_Marra_i
    deltas_RNM_i = results_RNM_i - results_NN_Marra_i

    deltas_SBR_t = results_SBR_t - results_NN_Marra_t
    deltas_RNM_t = results_RNM_t - results_NN_Marra_t

    r = np.arange(5) * 3.5 * barwidth
    ax.bar(r, means_deltas, width=barwidth, edgecolor='white', label='KENN')

    if other_deltas == 'i':
        r2 = [x + barwidth for x in r]
        r3 = [x + barwidth for x in r2]
        ax.bar(r2, deltas_SBR_i, width=barwidth,
                edgecolor='white', label='SBR')
        ax.bar(r3, deltas_RNM_i, width=barwidth,
                edgecolor='white', label='RNM')

    elif other_deltas == 't':
        r2 = [x + barwidth for x in r]
        r3 = [x + barwidth for x in r2]
        ax.bar(r2, deltas_SBR_t, width=barwidth, edgecolor='white', label='SBR')
        ax.bar(r3, deltas_RNM_t, width=barwidth, edgecolor='white', label='RNM')

    ax.set_xlabel('Percentage of Training', fontweight='bold')
    ax.set_xticks(r2,
               list(history.keys()))

    ax.set_title(title)


In [ ]:
fig, axes = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(15, 5))
fig.suptitle('Improvements over the NN', fontsize=16)
plot_deltas(history_inductive[2], axes[0], title='Inductive', other_deltas='i', barwidth=0.6)
axes[0].set_ylabel('Improvement in accuracy', fontweight='bold')
plot_deltas(history_transductive[2], axes[1], title='Transductive', other_deltas='t', barwidth=0.6)
axes[1].legend(loc='upper right')

fig.savefig('plots/deltas.png')
fig.show()

# Plot accuracies with different number of KEs

In [ ]:
color_NN = 'grey'
colors_map = ['darkblue', 'darkorange', 'darkgreen', 'darkred', 'lightblue', 'grey', 'brown']
color_SBR = ['pink']
color_RNM = ['purple']

In [ ]:
def plot_accuracies_n_layers(history_all, ax, title, confidence_level=0.99):
    n_layers_tried = len(history_all)
    
    means_kenn_list = []
    stds_kenn_list = []
    confidence_margins_kenn_list = []
    for history in history_all:
        confidence_margins_nn, confidence_margins_kenn, _ = get_all_confidence_margins(history, confidence_level)
        means, stds, means_kenn, stds_kenn, _, _ = get_means_and_stds(history)

        confidence_margins_kenn_list.append(confidence_margins_kenn)
        means_kenn_list.append(means_kenn)
        stds_kenn_list.append(stds_kenn)

    
    means_matrix = np.stack([means] + [np.array(mean) for mean in means_kenn_list], 0).T
    confidence_margins_matrix = np.stack([confidence_margins_nn] + [np.array(cm) for cm in confidence_margins_kenn_list], 0).T

    r = np.array([0,1,2,3,4,5,6])

    training_percentage = ['10%', '25%', '50%', '75%', '90%']
    colors = ['grey', 'darkblue', 'orange', 'darkgreen', 'darkred']
    for tp, means, confidence_margins, c in zip(training_percentage, means_matrix, confidence_margins_matrix, colors):
        ax.plot(r, means, '-+', label=tp, color=c)
        ax.fill_between(r, [m - cm for m, cm in zip(means, confidence_margins)],
                           [m + cm for m, cm in zip(means, confidence_margins)], color=c, alpha=.15)

    # Add xticks on the middle of the group bars
    ax.set_xlabel('Number of KEs', fontweight='bold')
    ax.set_title(title, fontsize=13)

    


fig, axes = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(15, 5))
fig.suptitle('Accuracy results at varying number of KE layers (with 99% confidence intervals)', fontsize=16)
plot_accuracies_n_layers(history_inductive, axes[0], 'Inductive')
axes[0].set_ylabel('Test Accuracy', fontweight='bold')
plot_accuracies_n_layers(history_transductive, axes[1], 'Transductive')
axes[1].legend(loc='best')


fig.savefig('plots/n_layers_plot.png')
fig.show()

In [ ]:
# Histograms

In [ ]:
plot_histograms(history_inductive[2], 'history_inductive', bw=0.6, bins=40)


In [ ]:
plot_histograms(history_transductive[2], 'history_transductive', bw=0.6, bins=40)

# Plot execution time

In [ ]:
def plot_time(history_all, ax, x_label, title, others, barwidth=0.1, confidence_level=0.95):
    
    time = []
    for p in ['10', '25', '50', '75', '90']:
        time.append(np.mean([a['time'] for a in history_all[0][p]['NN']]))

    ax.bar(0 * barwidth, np.mean(time), label='KENN l={}'.format(0), width=barwidth, color=color_NN, edgecolor='white')
    print(np.mean(time))
    
    for i in range(3):
        time = []
        for p in ['10', '25', '50', '75', '90']:
            time.append(np.mean([a['time'] for a in history_all[i][p]['KENN']]))

        print('{}:{} -> {}'.format(i + 1, i * barwidth, np.mean(time)))
        ax.bar((i + 1) * barwidth, np.mean(time), label='KENN l={}'.format(i + 1), width=barwidth, color=colors_map[i], edgecolor='white')
    
    if others:
        ax.bar(4 * barwidth, 87.35614902210236, label='SBR', width=barwidth, color=color_SBR, edgecolor='white')
        ax.bar(5 * barwidth, 215.69408444309235, label='RNM', width=barwidth, color=color_RNM, edgecolor='white')
    
    if others:
        ax.set_xticks([i * barwidth for i in range(6)], [0, 1, 2, 3,'SBR','RNM'])
    else:
        ax.set_xticks([i * barwidth for i in range(4)], [0, 1, 2, 3])


    # Add xticks on the middle of the group bars
    ax.set_xlabel(x_label, fontweight='bold')
    ax.set_title(title, fontsize=13)

    


fig, axes = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(15, 5))
fig.suptitle('Execution time', fontsize=16)
plot_time(history_inductive, axes[0], 'Number of KEs', 'Inductive', False)
axes[0].set_ylabel('Time (s)', fontweight='bold')
plot_time(history_transductive, axes[1], 'Number of KEs', 'Transductive', False)

fig.savefig('plots/time_kenn.png')
fig.show()


In [ ]:
fig, axes = plt.subplots(1, 2, sharex='col', sharey='row', figsize=(15, 5))
fig.suptitle('Execution time', fontsize=16)
plot_time(history_inductive, axes[0], 'Number of KEs', 'Inductive', False)
axes[0].set_ylabel('Time (s)', fontweight='bold')
plot_time(history_transductive, axes[1], 'Model', 'Transductive', True)
axes[1].set_yscale('log')
# axes[0].legend(loc='upper left')
    
fig.savefig('plots/time_all.png')
fig.show()